In [1]:
import glob
import multiprocessing
import os
import sys
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from pandas.core.common import flatten
import uuid
from tqdm import tqdm
from datetime import datetime
import json

import pandas as pd
from pandas.core.frame import DataFrame

OUTPUT_PATH="data/measurements"

In [2]:
files = os.listdir(OUTPUT_PATH)
files

['2021-34-336.parquet',
 '2021-30-21.parquet',
 '2021-31-336.parquet',
 '2021-40-9.parquet',
 '2021-32-336.parquet',
 '2021-35-336.parquet',
 '2021-38-336.parquet',
 '2021-39-336.parquet',
 '2021-33-336.parquet',
 '2021-36-336.parquet',
 '2021-37-336.parquet']

In [3]:
parquetfp = os.path.join(OUTPUT_PATH, files[1])

In [4]:
df = pd.read_parquet(parquetfp)
df.describe()

,azure_brazilsouth,gcp_southamerica-east1,aws_sa-east-1,aws_ca-central-1,gcp_northamerica-northeast1,aws_us-east-1,azure_eastus,gcp_us-east4,aws_us-west-1,aws_us-west-2,...,gcp_europe-west3,aws_ap-south-1,gcp_asia-south1,azure_centralindia,aws_ap-northeast-1,gcp_asia-northeast1,azure_japaneast,aws_ap-southeast-2,azure_australiaeast,gcp_australia-southeast1
count,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,...,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [76]:
# data_pr = (df['provider'] + "_" + df['region']).unique()
# data_pr.sort()
# data_pr, len(data_pr)

(array(['AWS_ap-northeast-1', 'AWS_ap-south-1', 'AWS_ap-southeast-2',
        'AWS_ca-central-1', 'AWS_eu-central-1', 'AWS_eu-west-2',
        'AWS_sa-east-1', 'AWS_us-east-1', 'AWS_us-west-1', 'AWS_us-west-2',
        'AZURE_australiaeast', 'AZURE_brazilsouth', 'AZURE_centralindia',
        'AZURE_eastus', 'AZURE_germanywestcentral', 'AZURE_japaneast',
        'AZURE_uksouth', 'AZURE_westus', 'AZURE_westus2',
        'GCP_asia-northeast1', 'GCP_asia-south1',
        'GCP_australia-southeast1', 'GCP_europe-west2', 'GCP_europe-west3',
        'GCP_northamerica-northeast1', 'GCP_southamerica-east1',
        'GCP_us-east4', 'GCP_us-west2', 'GCP_us-west4'], dtype=object),
 29)

In [5]:
df = pd.read_parquet("data/mstatsdf.parquet")
df.describe()

,azure_brazilsouth,gcp_southamerica-east1,aws_sa-east-1,aws_ca-central-1,gcp_northamerica-northeast1,aws_us-east-1,azure_eastus,gcp_us-east4,aws_us-west-1,aws_us-west-2,...,gcp_europe-west3,aws_ap-south-1,gcp_asia-south1,azure_centralindia,aws_ap-northeast-1,gcp_asia-northeast1,azure_japaneast,aws_ap-southeast-2,azure_australiaeast,gcp_australia-southeast1
count,357.000000,357.0,357.000000,357.0,357.0,357.000000,357.000000,357.0,357.0,357.000000,...,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.000000,357.0
mean,0.994398,1.0,1.002801,1.0,1.0,0.977591,1.002801,1.0,1.0,0.994398,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.985994,1.0
std,0.074743,0.0,0.052926,0.0,0.0,0.148217,0.091756,0.0,0.0,0.074743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.117679,0.0
min,0.000000,1.0,1.000000,1.0,1.0,0.000000,0.000000,1.0,1.0,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,1.0
25%,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
50%,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
75%,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
max,1.000000,1.0,2.000000,1.0,1.0,1.000000,2.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
